In [10]:
import sys
print(sys.version)

3.12.9 | packaged by conda-forge | (main, Feb 14 2025, 08:00:06) [GCC 13.3.0]


In [11]:
import uproot
import awkward as ak
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os #for looping over files in a directory
import math
import pandas as pd
from matplotlib.ticker import ScalarFormatter
import json
from pathlib import Path
import vector

First load in a file, same file as we use for the other notebooks:

In [12]:
dataset_runnable = json.loads(Path("af_v2_onefile.json").read_text())
#print(dataset_runnable)
file_to_open=list(dataset_runnable['Znunugamma']['files'].keys())[0]
tree_name="analysis"

In [13]:
data=None
with uproot.open(file_to_open) as f:
    #print("Keys in the ROOT file:", f.keys())
    #tree = f['analysis;1']
    #tree.show("name")

    tree_name = list(f.keys())[0]  # Get the first available tree
    tree = f[tree_name]  # Load the tree

    # Print only the variable names
    #print("\n".join(tree.keys()))
    data=tree.arrays()

Now apply some basic event-level selections, and keep the subset of events passing all selections.  There are some debugging
lines where I figure out how many events are left after different stages of selection, to compare with the eventloop example.

In [148]:
met_mask=(data.met_met_NOSYS<250000.)
el_mask=(ak.sum(data.el_pt_NOSYS,axis=1)==0)
mu_mask=(ak.sum(data.mu_pt_NOSYS,axis=1)==0)
vgam_mask=(data.in_vgamma_overlap_7>0)
leadjet_mask=(ak.fill_none(ak.max(data.jet_pt_NOSYS,axis=1),0)>=100000.)
btag_mask=(ak.sum(data.jet_select_btag_NOSYS,axis=1)==0)

print(f"all             {len(data)}")

# mindphi requirement
met=vector.zip({'pt': data.met_met_NOSYS, 'eta': np.zeros(len(data)), 'phi': data.met_phi_NOSYS, 'mass': np.zeros(len(data))})
jets = vector.zip({'pt': data.jet_pt_NOSYS, 'eta': data.jet_eta, 'phi': data.jet_phi, 'mass': data.jet_m_NOSYS})
mindphi_mask=(ak.fill_none(ak.min(jets.deltaphi(met),axis=1),0)>0.4)

presel_data=data[met_mask & el_mask & mu_mask & vgam_mask & leadjet_mask & btag_mask & mindphi_mask]
print(f"post mindphi    {len(presel_data.met_met_NOSYS)}")

all             872532
post mindphi    14790


The cutflow matches the eventloop numbers.

Now select events that have at least one "good" photon that passes basic quality criteria.

In [149]:
ph_preselection=((presel_data.ph_pt_NOSYS>10000) & 
                 ((abs(presel_data.ph_eta)<1.37) | ((abs(presel_data.ph_eta)>1.52) & 
                                                    (abs(presel_data.ph_eta)<2.37))) &
                 (presel_data.ph_select_or_dR02Ph_NOSYS==1) &
                 (((presel_data.ph_isEM_NOSYS)&0x45fc01)==0) &
                 (presel_data.ph_select_baseline_NOSYS==1)
                )

# keep events that have at least one photon that passes all photon preselection cuts
ph_presel_data=presel_data[(ak.any(ph_preselection,axis=1)) & (ak.fill_none(ak.max(presel_data.ph_pt_NOSYS,axis=1),0)>10000.)]

print(len(ph_presel_data))

12072


This is also consistent with eventloop example.

Now I'm going to try to apply all the photon cuts.  Ideally I'd like to be able to pick out a photon with some preselection criteria, and then categorize it as A/B/C/D/none based on whether it passes some other cuts.  Still working on that....

In [235]:
# define tight and loose cuts
ph_preselection=((ph_presel_data.ph_pt_NOSYS>10000) & 
                 ((abs(ph_presel_data.ph_eta)<1.37) | ((abs(ph_presel_data.ph_eta)>1.52) & 
                                                       (abs(ph_presel_data.ph_eta)<2.37))) &
                 (ph_presel_data.ph_select_or_dR02Ph_NOSYS==1) &
                 ((ph_presel_data.ph_isEM_NOSYS&0x45fc01)==0) &
                 (ph_presel_data.ph_select_baseline_NOSYS==1)
                )

print(repr(ph_preselection))
print(repr(ph_presel_data))

print(ph_presel_data[ak.any(ph_preselection, axis=1)])

print(ph_presel_data.ph_eta[ph_preselection])

print(ph_preselection)
print(ak.argsort(ph_preselection))
#indices=ak.argsort(ph_preselection)
indices=ak.unflatten(ak.argmax(ph_preselection,axis=1),1)
print()
print(indices)
#for i in range(100):
#    print(f"{i:2d} {indices[i]}")

print(ph_preselection)
print(ph_preselection[25])
print(indices[25])

#print(ph_presel_data.ph_pt_NOSYS[indices])
#print(ph_presel_data.ph_pt_NOSYS[indices][:,0])

# maybe this kind of thing will be useful someday.
#print(ak.where(ph_preselection,ph_presel_data.ph_select_tightID_NOSYS,0))

#print(ak.firsts(ak.argsort(ph_preselection.ph_pt_NOSYS,ascending=False)))

<Array [[True], [True], [...], ..., [True], [True]] type='12072 * var * bool'>
<Array [{...}, {...}, {...}, ..., {...}, {...}] type='12072 * {actualIntera...'>
[{actualInteractionsPerCrossing: 32.5, ...}, {...}, {...}, ..., {...}, {...}]
[[1.02], [1.31], [-0.713, 1.17], [2.1], ..., [0.957], [-0.88], [1.98], [0.448]]
[[True], [True], [True, True], [True], ..., [True], [True], [True], [True]]
[[0], [0], [0, 1], [0], [0], [0], [0], ..., [0], [0], [0], [0], [0], [0], [0]]

[[0], [0], [0], [0], [0], [0], [0], [0], ..., [0], [0], [0], [0], [0], [0], [0]]
[[True], [True], [True, True], [True], ..., [True], [True], [True], [True]]
[False, True]
[1]


In [236]:
ph_tight=(ak.firsts(ph_presel_data.ph_select_tightID_NOSYS[indices])>0)
print(ph_tight)
print(len(ph_tight))


print(ph_presel_data.ph_select_tightIso_NOSYS[indices])
ph_iso=(ak.firsts(ph_presel_data.ph_select_tightIso_NOSYS[indices])>0)
print(ph_iso)

ph_truth=((ak.firsts(ph_presel_data.ph_truthType[indices])!=16) & 
          (ak.firsts(ph_presel_data.ph_truthType[indices])!=0))
print(ph_truth)

print(ph_presel_data.ph_pt_NOSYS[indices][:,0])
print(len(ph_presel_data.ph_pt_NOSYS[indices][:,0]))

print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight ][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_iso   ][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_truth ][:,0]))

print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight &  ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight & ~ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight &  ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight & ~ph_iso &  ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight &  ph_iso & ~ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight & ~ph_iso & ~ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight &  ph_iso & ~ph_truth][:,0]))
print(len(ph_presel_data.ph_pt_NOSYS[indices][~ph_tight & ~ph_iso & ~ph_truth][:,0]))
#print(ak.num(ph_tight_iso_truth_events,axis=0))
#print(ak.firsts(ph_tight_iso_truth_events.ph_pt_NOSYS))

[True, True, True, True, True, True, ..., True, True, True, False, True, False]
12072
[[0], [0], [1], [1], [1], [1], [1], [1], ..., [1], [1], [0], [0], [0], [1], [1]]
[False, False, True, True, True, True, ..., False, False, False, True, True]
[True, True, True, True, True, True, ..., True, True, True, True, True, True]
[1.53e+04, 2.95e+04, 2.44e+05, 2.37e+05, ..., 1.61e+05, 2.12e+05, 2.36e+05]
12072
11428
8838
11944
8461
2915
339
229
18
34
20
56


In [128]:
8175+2762+332+247+89+80+145+242

12072

The 8175 number doesn't match the eventloop example, it's off by a few hundred, something is wrong.

In [130]:
8175+2762+332+247

11516

In [211]:
#photons = vector.zip({k.split('_')[1:]: ph_presel_data[k] for k in ak.fields(ph_presel_data) if k.startswith('ph_')})
photons = vector.zip({'pt': ph_presel_data.ph_pt_NOSYS, 'eta': ph_presel_data.ph_eta, 'phi': ph_presel_data.ph_phi, 'mass': 0,
                      'isEM': ph_presel_data.ph_isEM_NOSYS, 'tight': ph_presel_data.ph_select_tightID_NOSYS,
                      'iso': ph_presel_data.ph_select_tightIso_NOSYS, 'truthType': ph_presel_data.ph_truthType,
                      'baseline': ph_presel_data.ph_select_baseline_NOSYS, 'OR': ph_presel_data.ph_select_or_dR02Ph_NOSYS})
ak.fields(photons)

['rho',
 'phi',
 'eta',
 'tau',
 'isEM',
 'tight',
 'iso',
 'truthType',
 'baseline',
 'OR']

In [214]:
ph_tight=(ak.firsts(photons[indices].tight)>0)
print(ph_tight)
print(len(ph_tight))
ph_iso=(ak.firsts(photons[indices].iso)>0)
print(ph_iso)
ph_truth=((ak.firsts(photons[indices].truthType)!=16) & 
          (ak.firsts(photons[indices].truthType)!=0))
print(ph_truth)

ph_tight_iso_truth=(ph_tight & ph_iso & ph_truth)
print(ph_tight_iso_truth)

print(ph_presel_data.ph_pt_NOSYS[indices][:,0])
print(len(ph_presel_data.ph_pt_NOSYS[indices][:,0]))
 

print(ph_presel_data.ph_pt_NOSYS[indices][ph_tight_iso_truth][:,0])
print(len(ph_presel_data.ph_pt_NOSYS[indices][ ph_tight_iso_truth][:,0]))

print(None in ph_presel_data.ph_pt_NOSYS[indices][ ph_tight_iso_truth][:,0])


print(indices)
for i in range(100):
    print(f"{i:2d} {indices[i]}")

print(photons[25].pt)
print(f"pt:    {photons[25].pt}")
print(f"isEM   {photons[25].isEM}")
print(f"tight  {photons[25].tight}")
print(f"iso    {photons[25].iso}")
print(f"eta    {photons[25].eta}")
print(f"phi    {photons[25].phi}")
print(f"OR     {photons[25].OR}")
print(f"truth  {photons[25].truthType}")

print(len(photons[indices][ ph_tight ][:,0]))
print(len(photons[indices][ ph_iso   ][:,0]))
print(len(photons[indices][ ph_truth ][:,0]))

print(len(photons[indices][ ph_tight &  ph_iso &  ph_truth][:,0].pt))
print(len(photons[indices][ ph_tight & ~ph_iso &  ph_truth][:,0].pt))
print(len(photons[indices][~ph_tight &  ph_iso &  ph_truth][:,0].pt))
print(len(photons[indices][~ph_tight & ~ph_iso &  ph_truth][:,0].pt))
print(len(photons[indices][ ph_tight &  ph_iso & ~ph_truth][:,0].pt))
print(len(photons[indices][ ph_tight & ~ph_iso & ~ph_truth][:,0].pt))
print(len(photons[indices][~ph_tight &  ph_iso & ~ph_truth][:,0].pt))
print(len(photons[indices][~ph_tight & ~ph_iso & ~ph_truth][:,0].pt))
#print(ak.num(ph_tight_iso_truth_events,axis=0))
#print(ak.firsts(ph_tight_iso_truth_events.ph_pt_NOSYS))

[True, True, True, True, True, True, ..., True, True, True, False, True, False]
12072
[False, False, True, True, True, True, ..., False, False, False, True, True]
[True, True, True, True, True, True, ..., True, True, True, True, True, True]
[False, False, True, True, True, True, ..., False, False, False, True, False]
[1.53e+04, 2.95e+04, 2.44e+05, 2.37e+05, ..., 1.61e+05, 2.12e+05, 2.36e+05]
12072
[2.44e+05, 2.37e+05, 2.18e+05, 2.05e+05, ..., 2.46e+05, 2.51e+05, 2.12e+05]
8175
False
[[0], [0], [0, 1], [0], [0], [0], [0], ..., [0], [0], [0], [0], [0], [0], [0]]
 0 [0]
 1 [0]
 2 [0, 1]
 3 [0]
 4 [0]
 5 [0]
 6 [0]
 7 [0]
 8 [0]
 9 [0]
10 [0]
11 [0]
12 [0]
13 [0]
14 [0]
15 [0]
16 [0]
17 [0]
18 [0]
19 [0]
20 [0]
21 [0]
22 [0]
23 [0]
24 [0]
25 [0, 1]
26 [0]
27 [0]
28 [0]
29 [0, 1]
30 [0]
31 [0]
32 [0]
33 [0]
34 [0]
35 [0]
36 [0]
37 [0]
38 [0]
39 [0]
40 [0]
41 [0]
42 [0]
43 [0]
44 [0]
45 [0]
46 [0]
47 [0]
48 [0]
49 [0]
50 [0]
51 [0]
52 [0]
53 [0]
54 [0]
55 [0]
56 [0]
57 [0]
58 [0]
59 [0]
60 [

In [215]:
print(f"{2102272:x}")

201400


In [216]:
print(f"{2102272&0x45fc01:x}")

1400
